In [ ]:
! MACOSX_DEPLOYMENT_TARGET=14.5 pip install --default-timeout=100 -r requirements2.txt


In [6]:
local_llm = "llama3"

# parse and store pdf data in vector stores (ChromaDB)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.document_loaders import FireCrawlLoader
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.docstore.document import Document

from langchain_community.embeddings.fastembed import FastEmbedEmbeddings


urls = [
    "https://confluence.ship.gov.sg/pages/viewpage.action?pageId=1050239830&preview=%2F1050239830%2F1252494500%2FHealthcare_7-Feb-2024.pdf"
]

docs = ["pdf/Healthcare_7-Feb-2024.pdf"]

#split documents
docs_list = [item for sublist in docs for item in sublist]

#split documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024, chunk_overlap=80, length_function=len, is_separator_regex=False
)

# doc_splits = text_splitter.split_documents(docs_list)

# #Filter out complex metadata and ensure proper document formatting
# filtered_docs = []
# for doc in doc_splits:
#     #ensure the doc is an instance of document and has a 'metadata' attribute
#     if isinstance(doc,Document) and has

# #add to vectorDB
# vectorstore = Chroma.from_documents(
#     documents = doc_splits,
#     collection_name = "rag-chroma",
#     embedding = GPT4AllEmbeddings(),
# )

# retriver = vectorstore.as_retriever()
embedding = FastEmbedEmbeddings()

vector_store = Chroma(persist_directory="db", embedding_function=embedding)

# print("creating chain")
retriever = vector_store.as_retriever(
)




In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

embedding = FastEmbedEmbeddings()

vector_store = Chroma(persist_directory="db", embedding_function=embedding)

# print("creating chain")
retriever = vector_store.as_retriever(
)

In [4]:
import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

# Step 1: Read the contents of documents.txt
with open('db/documents.txt', 'r') as file:
    documents = file.readlines()

# Ensure the documents are in the right format (list of strings)
documents = [doc.strip() for doc in documents]

# Step 2: Initialize the embedding model
embedding = FastEmbedEmbeddings()

# Step 3: Initialize the Chroma vector store with the embedding function
persist_directory = "db"
if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)

vector_store = Chroma(persist_directory=persist_directory, embedding_function=embedding)

# Step 4: Add documents to the vector store
for document in documents:
    vector_store.add_texts([document])

# Step 5: Create the retriever
retriever = vector_store.as_retriever()

print("Retriever created successfully!")


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Retriever created successfully!


In [ ]:
# Example usage of the retriever
query = "Your query here"
results = retrieve(query)
for result in results:
    print(result)

# Determine document relevance

In [19]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser

#LLM
llm = ChatOllama(model=local_llm, format="json", temperature = 0)

prompt= PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> 
    You are a grader assessing relevance of a retrieved document to a user question. 
    If the document contains keywords related to the user question, grade it as relevant.
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate wherther the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document}\n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question","document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()
question = "what the reason behind error code VCA373 and write a playbook to investigate an error"
error_code = "VCA373"
docs = retriever.invoke(question) + retriever.invoke(error_code)

# Debug print statements
print("Retrieved documents:")
print(docs)

for ele in docs:
    print(ele)

# Ensure we have the correct index and document content
if len(docs) > 1:
    doc_text = docs[1]
    print("Document text:")
    print(doc_text)

    # Invoke the grader
    result = retrieval_grader.invoke({"question": question, "document": doc_text})
    print("Grader result:")
    print(result)
else:
    print("Insufficient documents retrieved.")

Retrieved documents:
[Document(page_content="add VCA034 - INVALID REASON CODE to the claim level errors of 'the claim' ;"), Document(page_content="add VCA032 - INVALID SMC NUMBER to the claim level errors of 'the claim' ;"), Document(page_content='VCA009 - INVALID'), Document(page_content='VCA009 - INVALID'), Document(page_content='about:blank 373/495'), Document(page_content='about:blank 353/495'), Document(page_content='about:blank 463/495'), Document(page_content='about:blank 310/495')]
page_content="add VCA034 - INVALID REASON CODE to the claim level errors of 'the claim' ;"
page_content="add VCA032 - INVALID SMC NUMBER to the claim level errors of 'the claim' ;"
page_content='VCA009 - INVALID'
page_content='VCA009 - INVALID'
page_content='about:blank 373/495'
page_content='about:blank 353/495'
page_content='about:blank 463/495'
page_content='about:blank 310/495'
Document text:
page_content="add VCA032 - INVALID SMC NUMBER to the claim level errors of 'the claim' ;"
Grader result:


# Error identifier

In [67]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser

#LLM
llm = ChatOllama(model=local_llm, format="json", temperature = 0)

prompt= PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> 
    You are an indentifier.
    Identify the error code from the given question / sentence. An example could be VCA373 . Error codes usually start with a few capital alphabets followed by a few numbers
    It does not need to be a stringent test. The goal is to identify only the error code and return that. \n
    Return the error code if it is present, else return an empty string ''. \n
    Provide the error code as a JSON with a single key 'error_code' and no preamble or explanation.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the sentence: \n\n {question}\n\n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question"],
)

error_identifier = prompt | llm | JsonOutputParser()


def extract_error_code(question):
    error_code_JSON = error_identifier.invoke({"question": question})
    print(error_code_JSON)
    return error_code_JSON["error_code"]

### Document Retrieval and Combination
def retrieve_and_combine_documents(question, retriever):
    error_code = extract_error_code(question)
    combined_docs = []

    if error_code:
        print(f"Extracted Error Code: {error_code}")
        general_docs = retriever.get_relevant_documents(question)
        error_code_docs = retriever.get_relevant_documents(error_code)

        # Check for exact match of the error code and prioritize it
        exact_match_docs = [doc for doc in error_code_docs if error_code in doc.page_content]

        # Combine documents prioritizing the exact match
        combined_docs = exact_match_docs + [doc for doc in error_code_docs if doc not in exact_match_docs] + [doc for doc in general_docs if doc not in error_code_docs]

    else:
        print("No Error Code found in the question.")
        combined_docs = retriever.get_relevant_documents(question)

    return combined_docs[:5]

# Usage Example
question = "What is the reason behind error code CHC016?"
combined_docs = retrieve_and_combine_documents(question, retriever)

# Debug print statements
print("Retrieved documents:")
for i, doc in enumerate(combined_docs):
    print(f"combined doc {i}: {doc.page_content}")

# print(question)
# print(combined_docs)
# generation = rag_chain.invoke({"context": combined_docs, "question": question})
# print("Generated Response:")
# print(generation)


{'error_code': 'CHC016'}
Extracted Error Code: CHC016
Retrieved documents:
combined doc 0: TL0001 - Dental
combined doc 1: TL0001 - Dental
combined doc 2: TL0001 - Dental
combined doc 3: TL0001 - Dental
combined doc 4: invalid for your hospital code. For further


In [98]:
def extract_error_code(question):
    error_code_JSON = error_identifier.invoke({"question": question})
    return error_code_JSON["error_code"]

### Document Retrieval and Combination
def retrieve_and_combine_documents(question, retriever):
    error_code = extract_error_code(question)
    combined_docs = []

    if error_code:
        print(f"Extracted Error Code: {error_code}")

        with open('db/documents.txt', 'r') as file:
            lines = file.readlines()

        matched_snippets = set()
        skip_lines = 0
        skip = False
        for i, line in enumerate(lines):

            skip_lines -= 1

            if error_code in line and not skip:
                start = max(0, i - 10)
                end = min(len(lines), i + 11)
                snippet = ''.join(lines[start:end])
                matched_snippets.add(snippet)

                skip_lines = 10
                skip = True

                if len(matched_snippets) >= 5:
                    break

            if skip_lines <= 0:
                skip = False
                skip = 0



        combined_docs = list(matched_snippets)[:5]
        print("Exact match found in documents.txt")
        print("-------------------------------")
        for idx, doc in enumerate(combined_docs):
            print(f"combined_doc {idx}: {doc}")

    else:
        print("No Error Code found in the question.")
        # Retrieve documents only for the general question if no error code is found
        combined_docs = retriever(question)

    return combined_docs

# # Usage Example
# question = "What is the reason behind error code CHC016?"
# combined_docs = retrieve_and_combine_documents(question, retriever)


# Generate answer

In [18]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

prompt= PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> 
    You are an assistant for question-answering tasks. Often, the context might be in code or descriptions, do your best to format it nicely and then analyse them.
    You can try to identify the specific things mentioned in the question and work from there.
    After formatting it nicely into a human readable format, return it at the end in the format "Context: (your formatted context)"
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    If you dont know, also mention what you could discern specifically from the context as well as what you think you might need to answer the given question.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question","context"],
)

llm = ChatOllama(model=local_llm, temperature=0, callbacks=callback_manager)

#chain

rag_chain = prompt | llm | StrOutputParser()

#run
question = "What is the reason behind error CHC016"
combined_docs = retrieve_and_combine_documents(question, retriever)
# Extract text from combined documents
# context = "\n\n".join([doc for doc in combined_docs])

generation = rag_chain.invoke({"context": combined_docs, "question": question})
print(generation)

Extracted Error Code: CHC016
Exact match found in documents.txt
-------------------------------
combined_doc 0: about:blank 94/495
Rule: Common-ClaimLimitReached
Preconditions:
definitions
set 'fully utilised limit' to a limit in all limits for 'financing scheme to process' in 'the claim'
where the limit has reached for this limit ;
Definition:
Limit reached scheme Add error code Add scheme name for
external remark
1 MSHL-Brachytherapy - Brachytherapy
CHC016 - MediShield Life
claim limit has been
reached.
MSHL-Brachytherapy
2 MSHL-Chemo - Chemotherapy
CHC016 - MediShield Life
claim limit has been
reached.
MSHL-Chemo
3 MSHL-Chemotherapy Services - MSHL Chemotherapy Services
CHC016 - MediShield Life

combined_doc 1: CHC009 - MediShield / MediShield Life claimable amount
computed is less than or equal to the deductible: there is
no payout from MediShield / MediShield Life
APPROVE MEDISHIELD_LIFE
7 CHC011 - Pregnancy / Childbirth is not covered under
MediShield / MediShield Life REJECT MED

# Check for answer for hallucination

In [22]:

llm = ChatOllama(model=local_llm, format="json", temperature=0)

prompt= PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> 
    You are a grader assessing whether an answer is grounded in / supported by a set of facts.
    Give a binary score 'yes' or 'no' score to indicate whether the answer is grounded in / supported by a set of facts.
    Provide the binary score as a JSON with a single ey 'score' and no preamble or explanation.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the facts: 
    \n-------\n 
    {documents}
    \n-------\n
    Here is the answer: {generation} <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["generation","document"],
)

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})


{'score': 'no'}

# Answer grader

In [ ]:

llm = ChatOllama(model=local_llm, format="json", temperature=0)

prompt= PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> 
    You are a grader assessing whether an answer is addressing the question properly.
    Give a binary score 'yes' or 'no' score to indicate whether the answer addresses the question well.
    Provide the binary score as a JSON with a single ey 'score' and no preamble or explanation.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Here is the answer: {generation} <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["generation","question"],
)

answer_grader = prompt | llm | JsonOutputParser()
answer_grader.invoke({"question": question, "generation": generation})


# Lang graph - Setup states & nodes

In [ ]:
from typing_extensions import TypedDict
from typing import List
import time

###state

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]
    error_code: str


from langchain.schema import Document

###Nodes

def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents

    """
    start_time = time.time()

    print("---RETRIEVE---")
    question = state["question"]

    error_code_JSON = error_identifier.invoke({"question": question})
    error_code = error_code_JSON["error_code"]

    if error_code != '':
        print(f"Extracted Error Code: {error_code}")
        # # Retrieve documents for the general question
        # general_docs = retriever.invoke(question)
        
        # # Retrieve documents specifically for the error code
        # error_code_docs = retriever.invoke(error_code)

        # exact_match_docs1 = [doc for doc in general_docs if error_code in doc.page_content]
        # exact_match_docs2 = [doc for doc in error_code_docs if error_code in doc.page_content]
        # # Combine the results, ensuring error code documents are prioritized or marked
        # # combined_docs = error_code_docs + [doc for doc in general_docs if doc not in error_code_docs]

        # combined_docs = exact_match_docs1 + [doc for doc in general_docs if doc not in exact_match_docs1]

        # print(f"combined_docs: {combined_docs}")
        # print("-------------------------------")
        # print(f"exac 1: {exact_match_docs1}")
        # print("-------------------------------")
        # print(f"gen docs: {general_docs}")
        # print("-------------------------------")
        # print(f"exac 1: {exact_match_docs2}")
        # print("-------------------------------")
        # print(f"errcodedocs: {error_code_docs}")


        with open('db/documents.txt', 'r') as file:
                lines = file.readlines()

        matched_lines = []
        for i, line in enumerate(lines):
            if error_code in line:
                start = max(0, i - 20)
                end = min(len(lines), i + 21)
                matched_lines.extend(lines[start:end])
                
        if matched_lines:
            combined_docs = matched_lines
            print("Exact match found in documents.txt")
            print("-------------------------------")
            print(f"combined_docs: {combined_docs}")




    else:
        print("No Error Code found in the question.")
        # Retrieve documents only for the general question if no error code is found
        combined_docs = retriever.invoke(question)

    elapsed_time = time.time() - start_time
    print(f"Retrieve time: {elapsed_time:.2f} seconds")
    state["retrieve_time"] = elapsed_time
    return {"documents": combined_docs, "question": question, "error_code": error_code}

def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will ... (TBD)

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents
    """
    start_time = time.time()

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    
    question = state["question"]
    documents = state['documents']

    #score each doc
    filtered_docs = []
    for d in documents:
        print(f"doc: {d}")
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score['score']
        #document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            #then do not include the document in filtered_docs
            continue

    elapsed_time = time.time() - start_time
    print(f"Grade documents time: {elapsed_time:.2f} seconds")
    state["grade_documents_time"] = elapsed_time
    return {"documents": filtered_docs, "question": question}

 
def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    start_time = time.time()

    print("---GENERATING---")
    question = state["question"]
    documents = state["documents"]

    #RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})

    elapsed_time = time.time() - start_time
    print(f"Generate time: {elapsed_time:.2f} seconds")
    state["generate_time"] = elapsed_time
    return {"documents": documents, "question": question, "generation": generation}


def grade_generation_v_documents_and_question(state):
    """

    """
    start_time = time.time()
    print("---CHECK HALLUCINATION---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    grade = score['score']

    #check hallucination
    if grade == "yes":
        print("--DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        #check question-answering
        print("---GRADE GENERATION VS QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score['score']
        if grade == "yes":
            print("--DECISION: GENERATION ADDRESSES QUESTION---")
            elapsed_time = time.time() - start_time
            print(f"Grade generation time: {elapsed_time:.2f} seconds")
            state["grade_generation_time"] = elapsed_time
            return "useful"
        else:
            print("--DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            elapsed_time = time.time() - start_time
            print(f"Grade generation time: {elapsed_time:.2f} seconds")
            state["grade_generation_time"] = elapsed_time
            return "not useful"

    else:
        print("--DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        elapsed_time = time.time() - start_time
        print(f"Grade generation time: {elapsed_time:.2f} seconds")
        state["grade_generation_time"] = elapsed_time
        return "not supported"


from langgraph.graph import END, StateGraph
workflow = StateGraph(GraphState)

#define nodes
workflow.add_node("retrieve", retrieve)
# workflow.add_node("grade_documents", grade_documents)
workflow.add_node("generate", generate)

In [ ]:
#build graph
workflow.set_entry_point("retrieve")
# workflow.add_edge("retrieve", "grade_documents")
workflow.add_edge("retrieve", "generate")
# workflow.add_edge("grade_documents", "generate")

# workflow.add_conditional_edges(
#     "generate",
#     grade_generation_v_documents_and_question,
#     {
#         "not supported": "generate",
#         "useful": END,
#         # "not useful"
#     },
# )
workflow.add_edge("generate", END)


In [ ]:
#compile
app = workflow.compile()

#Test
from pprint import pprint
inputs = {"question": "What is the reason behind error code VCA373. Come up with a play book to invstigate this"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")

print(value["generation"])

In [ ]:
import gradio as gr
import io
import contextlib
import sys
from typing import Any, Dict
import time

app = workflow.compile()

### Gradio Chat Interface Integration

### ChatInterface function with streaming for generation output
def chat_predict(message, history, state):
    try:
        start_time = time.time()
        inputs = {"question": message}
        full_output = ""
        state = {"generation": ""}

        for output in app.stream(inputs):
            for key, value in output.items():
                if key == "generation":
                    full_output += value['generation'] + "\n"
                    state["generation"] += value['generation'] + "\n"
                    current_time = time.time() - start_time
                    yield full_output + f"Running time: {current_time:.2f} seconds\n"
        
        total_time = time.time() - start_time
        yield full_output + f"Total time: {total_time:.2f} seconds\n + {state['generation']}"
    except Exception as e:
        yield str(e)

### Combine the interfaces
chat_interface = gr.ChatInterface(
    fn=chat_predict,
    title="Ask Question",
    # inputs=["text", "state"],
    # outputs="text",
    # state="state"
)

### Launch the Gradio app
chat_interface.launch()

In [ ]:
# context generator

# Prompt template for formatting context
context_formatting_template = PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> 
    You are an assistant for formatting tasks. Do not alter the content or add in any new content.
    You are only in charge of reformatting the context given to you. 
    The context is parsed from a pdf and might be part of code, descriptions or tables. Do your best to format it nicely into markdown.
    Format the given "context" variable and return it in the format "Context: (formatted context)". 
    If no "context" variable is given, just return an empty string.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["context"],
)

llm = ChatOllama(model=local_llm, temperature=0, callbacks=callback_manager)
context_formatter = prompt | llm | StrOutputParser()



In [112]:
import gradio as gr

from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama


prompt_template= PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> 
    You are an assistant for question-answering tasks. Often, the context might be in code, descriptions or tables, do your best to format it nicely and then analyse them.
    You can try to identify the specific things mentioned in the question and work from there.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    If you dont know, also mention what you could discern specifically from the context as well as what you think you might need to answer the given question.
    
    
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question","context"],
)

# After formatting the variable "context" given to you nicely into a human readable format, return it at the end in the format "Context: (formatted context)". 
#     If no "context" variable is given, dont need to add anything.

# Prompt template for formatting context
context_formatting_template = PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> 
    You are an assistant for formatting tasks. Do not alter the content or add in any new content.
    You are only in charge of reformatting the context given to you. 
    The context is parsed from a pdf and might be part of code, descriptions or tables. Do your best to format it nicely into markdown.
    Format the given "context" variable and return it in the format "Context: (formatted context)". 
    If no "context" variable is given, just return an empty string.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["context"],
)

# Initialize the LLM
local_llm = "llama3"  # Assuming llama3 is already defined
llm = ChatOllama(model=local_llm, temperature=0, callbacks=callback_manager)
llm_formatter = ChatOllama(model=local_llm, temperature=0, callbacks=callback_manager)

def format_history(msg, history):
    chat_history = [{"role": "system", "content": "hi"}]
    for query, response in history:
        chat_history.append({"role": "user", "content": query})
        chat_history.append({"role": "assistant", "content": response})
    chat_history.append({"role": "user", "content": msg})
    return chat_history

def generate_response(message, history, validate=False, check_hallucination=False, check_context=False):
    if history is None:
        history = []

    validation_agent = retrieval_grader
    hallucination_agent = hallucination_grader

    # Retrieve and combine documents
    combined_docs = retrieve_and_combine_documents(message, retriever)
    combined_docs_string = "\n\n".join([str(doc) for doc in combined_docs])

    # Generate the response
    prompt = prompt_template.format(question=message, context=combined_docs_string)
    response = llm.stream(prompt)
    result = ""
    for partial_answer in response:
        result += partial_answer.content
        history[-1][1] = result
        yield history, gr.update()  # Yield the main response first

    # Perform validation check if requested
    if validate:
        documents = validation_agent.invoke({"question": message, "document": doc_text})
        validation_result = "Validation Check: Documents validated successfully."
        history.append(["Validation Check", validation_result])
        yield history, gr.update(), validation_result

    # Perform context check if requested
    if check_context:
        context_prompt = context_formatting_template.format(context=combined_docs_string)
        context_response = llm_formatter.stream(context_prompt)
        context_result = ""
        for partial_context in context_response:
            context_result += partial_context.content

        # print(1)
        # history.append(["Context Check", context_result])
            history[-1][1] = result
            yield history, gr.update()

    # Perform hallucination check if requested
    if check_hallucination:
        hallucination_result = hallucination_agent.invoke({"documents": combined_docs, "generation": result})
        if hallucination_result:
            hallucination_result_text = "\n\nHallucination Check:\n" + hallucination_result
            history.append(["Hallucination Check", hallucination_result_text])
            yield history, gr.update(), hallucination_result_text

    history[-1][1] = result
    print(2)
    yield history, gr.update()

def add_message(history, message):
    if message is not None:
        history.append([message, None])
    return history, gr.update(value="")

with gr.Blocks() as demo:
    chatbot = gr.Chatbot([], elem_id="chatbot", bubble_full_width=False)
    message_input = gr.Textbox(placeholder="Enter message...", show_label=False)
    context_checkbox = gr.Checkbox(label="Show Context")
    validate_checkbox = gr.Checkbox(label="Validate Documents")
    hallucination_checkbox = gr.Checkbox(label="Check Hallucination")
    clear_btn = gr.Button("Clear")
    state = gr.State([])  # Initialize state as an empty list to hold the chat history

    def gradio_chat_ollama(history, validate, check_hallucination, check_context):
        message = history[-1][0] if history else ""
        generator = generate_response(message, history, validate, check_hallucination, check_context)
        print("doneonenenneeoneo")
        for response in generator:
            print("response")
            yield response

    chat_msg = message_input.submit(add_message, [state, message_input], [state, chatbot]).then(
        gradio_chat_ollama, [state, validate_checkbox, hallucination_checkbox, context_checkbox], [chatbot, state]
    )
    clear_btn.click(lambda: [], None, chatbot, queue=False)  # Clear the chat

    demo.queue()
    demo.launch()

Running on local URL:  http://127.0.0.1:7941

To create a public link, set `share=True` in `launch()`.


Extracted Error Code: VCA373
Exact match found in documents.txt
-------------------------------
combined_doc 0: Project: Healthcare
Status: new
Tags: []
Folder Name: [Healthcare]\common\screening\claimitemlevel
Rule: COMMON-PredeliveryAppealHRNExemption
Definition:
if
'the current claim item' contains scheme MSV-Predelivery - Pre-delivery [MSV]
and it is not true that the appeal outcome of 'the claim' is hrn exempted
then
add VCA373 - MOH CLAIM EXEMPTION REQUIRED to the claim level errors of 'the claim' ;
Documentation:
This rule checks that if the claim is MSV-predelivery and the appeal outcome is not hrn exempted error VCA373 will be added to the claim.
Properties:
Active: true
Categories: []
Created By: kueh_thian_phin@tech.gov.sg
Created On: Wed Jun 02 16:33:30 GMT+08:00 2021
Folder: claimitemlevel
Group: Healthcare
Last Changed By: deepak.agrawal@cognizant.com

Based on the provided context, error VCA373 is related to a rule in the Healthcare project. The rule is called "COMMON-Pr

In [ ]:

# def gradio_chat_ollama(message, history, validate, check_hallucination):
#     return chat_ollama(message, history, validate, check_hallucination)

# with gr.Blocks() as demo:
#     chatbot = gr.Chatbot()
#     with gr.Row():
#         with gr.Column(scale=7):
#             textbox = gr.Textbox(placeholder="Example: What is error VCA373?", container=False)
#         with gr.Column(scale=3):
#             validate_checkbox = gr.Checkbox(label="Validate Documents")
#             hallucination_checkbox = gr.Checkbox(label="Check Hallucination")
#     clear_btn = gr.Button("Clear")

#     textbox.submit(gradio_chat_ollama, [textbox, chatbot, validate_checkbox, hallucination_checkbox], chatbot)
#     clear_btn.click(lambda: None, None, chatbot, queue=False)

#     demo.launch()


# # Define the Gradio interface
# with gr.Blocks() as demo:
#     gr.Markdown("""<h1><center>Chat with Ollama</center></h1>""")
#     chatbot = gr.Chatbot()
#     message = gr.Textbox(placeholder="Example: What is error VCA373?", container=False, scale=7)
#     validate_checkbox = gr.Checkbox(label="Validate Documents")
#     hallucination_checkbox = gr.Checkbox(label="Check Hallucination")
#     clear_btn = gr.Button("Clear")
#     state = gr.State()

#     def gradio_chat_ollama(message, history, validate, check_hallucination):
#         return chat_ollama(message, history, validate, check_hallucination)

#     message.submit(gradio_chat_ollama, inputs=[message, state, validate_checkbox, hallucination_checkbox], outputs=[chatbot, state])
#     clear_btn.click(lambda: None, None, chatbot, queue=False)

#     demo.launch()


# # Define the Gradio interface
# chatbot = gr.ChatInterface(
#     generate_response,
#     chatbot=gr.Chatbot(
#         avatar_images=["user.jpg", "chatbot.png"],
#         height="64vh"
#     ),
#     additional_inputs=[
#         gr.Textbox(
#             "Behave as if you are a professional writer.",
#             label="System Prompt"
#         )
#     ],
#     title="LLama-3 Chatbot using 'Ollama'",
#     description="Feel free to ask any question.",
#     theme="soft",
#     submit_btn="⬅ Send",
#     retry_btn="🔄 Regenerate Response",
#     undo_btn="↩ Delete Previous",
#     clear_btn="🗑️ Clear Chat"
# )

# chatbot.launch()

In [102]:
import gradio as gr
import asyncio
from datetime import datetime
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama

# Prompt template for formatting context
context_formatting_template = PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> 
    You are an assistant for formatting tasks. Do not alter the content or add in any new content.
    You are only in charge of reformatting the context given to you. 
    The context is parsed from a pdf and might be part of code, descriptions or tables. Do your best to format it nicely into markdown.
    Format the given "context" variable and return it in the format "Context: (formatted context)". 
    If no "context" variable is given, just return an empty string.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["context"],
)

# Prompt template for answering the question
prompt_template= PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> 
    You are an assistant for question-answering tasks. Often, the context might be in code or descriptions, do your best to analyse them.
    You can try to identify the specific things mentioned in the question and context and work from there.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    If you dont know, also mention what you could discern specifically from the context as well as what you think you might need to answer the given question.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question","context"],
)

# Initialize the LLMs
local_llm = "llama3"  # Assuming llama3 is already defined
llm_formatter = ChatOllama(model=local_llm, temperature=0, callbacks=callback_manager)
llm_answer = ChatOllama(model=local_llm, temperature=0, callbacks=callback_manager)

def format_history(msg, history):
    chat_history = [{"role": "system", "content": "hi"}]
    for query, response in history:
        chat_history.append({"role": "user", "content": query})
        chat_history.append({"role": "assistant", "content": response})
    chat_history.append({"role": "user", "content": msg})
    return chat_history

# Wrapper to convert a synchronous generator to an asynchronous generator
async def async_stream(sync_gen):
    for item in sync_gen:
        yield item

async def format_context(context, history):
    print(f"[{datetime.now()}] Starting context formatting")
    context_prompt = context_formatting_template.format(context=context)
    formatted_context_response = llm_formatter.stream(context_prompt)
    formatted_context = ""
    async for partial_response in async_stream(formatted_context_response):
        formatted_context += partial_response.content
        history[-1][1] = f"Formatted Context: {formatted_context}"
        yield history, gr.update()
    print(f"[{datetime.now()}] Finished context formatting")

async def generate_answer(question, context, history):
    print(f"[{datetime.now()}] Starting answer generation")
    prompt = prompt_template.format(question=question, context=context)
    response = llm_answer.stream(prompt)
    result = ""
    async for partial_answer in async_stream(response):
        result += partial_answer.content
        history.append([f"Generated Answer: {result}", None])
        yield history, gr.update()
    print(f"[{datetime.now()}] Finished answer generation")

async def run_tasks_concurrently(history, combined_docs_string, question):
    format_task = asyncio.create_task(format_context(combined_docs_string, history))
    answer_task = asyncio.create_task(generate_answer(question, combined_docs_string, history))
    await asyncio.gather(format_task, answer_task)

def add_message(history, message):
    if message is not None:
        history.append([message, None])
    return history, gr.update(value=""), gr.update(value="")

with gr.Blocks() as demo:
    chatbot = gr.Chatbot([], elem_id="chatbot", bubble_full_width=False)
    message_input = gr.Textbox(placeholder="Enter message...", show_label=False)
    validate_checkbox = gr.Checkbox(label="Validate Documents")
    hallucination_checkbox = gr.Checkbox(label="Check Hallucination")
    clear_btn = gr.Button("Clear")
    state = gr.State([])  # Initialize state as an empty list to hold the chat history

    async def gradio_chat_ollama(history, validate, check_hallucination):
        message = history[-1][0] if history else ""
        print(f"Message received in gradio_chat_ollama: {message}")  # Debugging statement

        combined_docs = retrieve_and_combine_documents(message, None)
        combined_docs_string = "\n\n".join([str(doc) for doc in combined_docs])

        await run_tasks_concurrently(history, combined_docs_string, message)

    chat_msg = message_input.submit(add_message, [state, message_input], [state, chatbot, message_input]).then(
        gradio_chat_ollama, [state, validate_checkbox, hallucination_checkbox], [chatbot, state]
    )
    clear_btn.click(lambda: [], None, chatbot, queue=False)  # Clear the chat

    demo.queue()
    demo.launch()

Running on local URL:  http://127.0.0.1:7931

To create a public link, set `share=True` in `launch()`.


Message received in gradio_chat_ollama: What is error VCA373?
Extracted Error Code: VCA373
Exact match found in documents.txt
-------------------------------
combined_doc 0: Project: Healthcare
Status: new
Tags: []
Folder Name: [Healthcare]\common\screening\claimitemlevel
Rule: COMMON-PredeliveryAppealHRNExemption
Definition:
if
'the current claim item' contains scheme MSV-Predelivery - Pre-delivery [MSV]
and it is not true that the appeal outcome of 'the claim' is hrn exempted
then
add VCA373 - MOH CLAIM EXEMPTION REQUIRED to the claim level errors of 'the claim' ;
Documentation:
This rule checks that if the claim is MSV-predelivery and the appeal outcome is not hrn exempted error VCA373 will be added to the claim.
Properties:
Active: true
Categories: []
Created By: kueh_thian_phin@tech.gov.sg
Created On: Wed Jun 02 16:33:30 GMT+08:00 2021
Folder: claimitemlevel
Group: Healthcare
Last Changed By: deepak.agrawal@cognizant.com



Traceback (most recent call last):
  File "/Users/admin/Library/Python/3.9/lib/python/site-packages/gradio/queueing.py", line 521, in process_events
    response = await route_utils.call_process_api(
  File "/Users/admin/Library/Python/3.9/lib/python/site-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/admin/Library/Python/3.9/lib/python/site-packages/gradio/blocks.py", line 1935, in process_api
    result = await self.call_function(
  File "/Users/admin/Library/Python/3.9/lib/python/site-packages/gradio/blocks.py", line 1511, in call_function
    prediction = await fn(*processed_input)
  File "/Users/admin/Library/Python/3.9/lib/python/site-packages/gradio/utils.py", line 799, in async_wrapper
    response = await f(*args, **kwargs)
  File "/var/folders/mt/wb72h12n3yz8mzp138cssdw40000gn/T/ipykernel_17477/2056331426.py", line 102, in gradio_chat_ollama
    await run_tasks_concurrently(history, combined_docs_